In [129]:
import xarray as xr
import gdal
import numpy as np
import netCDF4 as nc4

In [130]:
%matplotlib inline
from __future__ import print_function
import time
from scipy.signal import fftconvolve
import matplotlib.pyplot as plt
import time

In [131]:
#--------------------
# function to make kernels and grid
#--------------------
# DOCSTRING
"""Arguments: kernel_x = kernel cells in x axis
            kernel_y = kernel cells on y axis
            grid_x = emissions grid cells on x
            grid_y = emissions grid cells on y
"""

def kernel_grid(kernel_x, kernel_y, grid_x, grid_y):
    big_array = np.random.random((grid_x, grid_y))
    small_array = np.random.random((kernel_x, kernel_y))
    convolved = fftconvolve(big_array, small_array)
    return convolved

In [132]:
#-------------------
# set up typical case
# 55 x 55, 25 million receptors
#-------------------
start = time.time()                         # start timer
test1 = kernel_grid(55, 55, 1000, 1000)   # set up grid conditions
test1_time = str((time.time()-start))       # run convolution function

# clear out old runs from memory
#test1 = None # keep it in memory for now while we test the netcdf compilation

print(str(test1_time + " seconds"))         # return the time data

0.251956939697 seconds


In [133]:
# Put test 1 in a netcdf file

# get dimensions for the array
x_coords = range(test1.shape[0])
y_coords = range(test1.shape[1])
# convert the numpy array to a DataArray
test1_array = xr.DataArray(test1, coords=[x_coords, y_coords], dims=['x', 'y'])

In [134]:
# make a netcdf file to store the data
f = nc4.Dataset('test_netcdf_data_store.nc','w', format='NETCDF4', zlib=True, least_significant_digit=1) #'w' stands for write

In [135]:
test1_array.to_netcdf('test_netcdf_data_store.nc', mode='a', group = 'test_1')

In [136]:
test2_array = test1_array * 5

In [137]:
test2_array.to_netcdf('test_netcdf_data_store.nc', mode='a', group = 'test_2')

In [138]:
test3_array = test2_array * 0.188283882

In [139]:
test3_array.to_netcdf('test_netcdf_data_store.nc', mode='a', group = 'test_3')

In [140]:
test4_array = test3_array * 0.188283882

In [141]:
test4_array.to_netcdf('test_netcdf_data_store.nc', mode='a', group = 'test_4')

In [142]:
# read the netcdf back in and have a look at it
f = nc4.Dataset('test_netcdf_data_store.nc','a')

grid_group = f.groups['test_1']

print("meta data for the dataset:")
print(f)
print("meta data for the test_1 group:")
print(f.groups['test_1'])
print(f.groups['test_1'].variables.keys()) # get list of variables by keys

meta data for the dataset:
<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: test_1, test_2, test_3, test_4

meta data for the test_1 group:
<type 'netCDF4._netCDF4.Group'>
group /test_1:
    dimensions(sizes): x(1054), y(1054)
    variables(dimensions): int64 x(x), int64 y(y), float64 __xarray_dataarray_variable__(x,y)
    groups: 

[u'x', u'y', u'__xarray_dataarray_variable__']


In [147]:
# get time in days since Jan 01,01
from datetime import datetime
today = datetime.today()

#Add global attributes
f.description = "Test combining several numpy arrays in a single netcdf file"
f.history = "Created " + today.strftime("%d/%m/%y")

x = grid_group.createVariable('X', 'f4', 'x')
y = grid_group.createVariable('Y', 'f4', 'y')

data = grid_group.createVariable('Concs', 'f4', ('x', 'y'), zlib=True, least_significant_digit=1)


#Add local attributes to variable instances
x.units = 'metres'
y.units = 'metres'
data.units = 'micrograms per metre cubed'
data.description = 'This is a description of the input data, could be as long as we want yada yada yada'

In [148]:
# read the netcdf back in and have a look at it
f = nc4.Dataset('test_netcdf_data_store.nc','r')

grid_group = f.groups['test_1']

print("meta data for the dataset:")
print(f)
print("meta data for the test_1 group:")
print(f.groups['test_1'])
print(f.groups['test_1'].variables.keys()) # get list of variables by keys

meta data for the dataset:
<type 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): 
    variables(dimensions): 
    groups: test_1, test_2, test_3, test_4

meta data for the test_1 group:
<type 'netCDF4._netCDF4.Group'>
group /test_1:
    dimensions(sizes): x(1054), y(1054)
    variables(dimensions): int64 x(x), int64 y(y), float64 __xarray_dataarray_variable__(x,y)
    groups: 

[u'x', u'y', u'__xarray_dataarray_variable__']
